---Sentiment analysis---
The script conducts sentiment analysis of news articles in the MIND dataset. It takes the PerSenT dataset for training, where articles are marked as postive, negative, or neutral, and applies this classification to the MIND articles. The classification is based on OpenAI's embeddings (text-embedding-ada-002). 


In [2]:
# imports
import pandas as pd
import openai
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from ast import literal_eval

In [7]:
# load data 
sent_news = pd.read_csv("sentiment_analysis/PerSenT/train.csv", sep=';') #document with user interactions
sent_news['CONTENT'] = sent_news['TITLE'].fillna('') + ' ' + sent_news['DOCUMENT'].fillna('')
sent_news_sorted = sent_news.drop(['DOCUMENT_INDEX', 'TARGET_ENTITY', 'TITLE', 'DOCUMENT'], axis=1)
sent_news_sorted.head()

,TRUE_SENTIMENT,CONTENT
0,Negative,German bank LBBW wins EU bailout approval Germ...
1,Positive,8th LD Writethru: 9th passenger released from ...
2,Negative,Commission: Bar Liberian president from office...
3,Neutral,AP Exclusive: Network flaw causes scary Web er...
4,Positive,Holyfield ' s wife says boxer hit her several ...
